In [9]:
import cdsapi
import os
from tqdm import tqdm
import time 

In [10]:
# Initialize CDS API client
c = cdsapi.Client()

In [11]:
# Define countries and their bounding boxes
countries_dict = {
    "SA": "Saudi Arabia",
    "YE": "Yemen",
    "OM": "Oman",
    "QA": "Qatar",
    "BH": "Bahrain",
    "AE": "United Arab Emirates",
    "KW": "Kuwait"
}

In [12]:
country_areas = {
    "SA": [16.0, 34.5, 32.0, 55.6667], # Saudi Arabia
    "YE": [12.1111, 41.833, 18.999, 54.5303],  # Yemen
    "OM": [16.642, 52.0, 26.4, 59.8],  # Oman
    "QA": [24.4814, 50.757, 26.1546, 51.6369],  # Qatar
    "BH": [25.5350, 50.454, 26.2640, 50.6640],  # Bahrain
    "AE": [22.6333, 51.5833, 26.084, 56.3817],  # United Arab Emirates
    "KW": [28.5243, 46.5527, 30.1037, 48.4315],  # Kuwait

}

In [13]:
# Define variables for surface and pressure levels
surface_variables = ['2m_temperature', #t2m
                    '2m_dewpoint_temperature', #d2m
                    '10m_u_component_of_wind', #u10
                    '10m_v_component_of_wind', #v10
                    'surface_solar_radiation_downwards', #ssrd
                    'surface_thermal_radiation_downwards', #strd
                    'evaporation', #e
                    'potential_evaporation', #pev
                    'sea_surface_temperature', #sst
                    'soil_temperature_level_1', #stl1
                    'soil_temperature_level_2', #stl2
                    'soil_temperature_level_3', #stl3
                    'soil_temperature_level_4', #stl4
                    'volumetric_soil_water_layer_1', #swvl1
                    'volumetric_soil_water_layer_2', #swvl2
                    'volumetric_soil_water_layer_3', #swvl3
                    'volumetric_soil_water_layer_4', #swvl4
                    'leaf_area_index_high_vegetation', #lai_hv
                    'leaf_area_index_low_vegetation', #lai_lv
                    'surface_pressure', #sp
                    'mean_sea_level_pressure', #msl
                    'convective_available_potential_energy', #cape
                    'geopotential', #z
                    'instantaneous_10m_wind_gust', #i10fg
                    'total_precipitation', #tp
                    'k_index' #kx
                    ]

pressure_level_variables = ['geopotential', #z
                            'relative_humidity', #r 
                            'temperature', #t
                            'u_component_of_wind',  #u
                            'v_component_of_wind', #v
                            'vertical_velocity', #w
                            'vorticity' #vo
                            ]


In [14]:
# Output directory
output_directory = r"Z:\Thesis\Data\Met\ERA5"

In [15]:
def download_data(year, country_code, country_name, area, output_directory):
    for variable_type, variables in [("surface", surface_variables), ("pressure", pressure_level_variables)]:
        output_path = os.path.join(output_directory, year, country_name, variable_type)
        os.makedirs(output_path, exist_ok=True)

        for variable in variables:
            file_name = f"{year}_{country_name}_{variable_type}_{variable}.nc"
            request = {
                'product_type': 'reanalysis',
                'format': 'netcdf',
                'area': area,
                'variable': variable,
                'year': year,
                'month': [f"{i:02d}" for i in range(1, 13)],
                'day': [f"{i:02d}" for i in range(1, 31)],
                'time': [f"{i:02d}:00" for i in range(0, 24, 3)],
            }
            if variable_type == 'pressure':
                request.update({'pressure_level': ['1000', '925', '850', '700', '500', '300', '200', '100', '50', '10']})

            print(f"Downloading {file_name} for {country_name}, {year}...")
            c.retrieve(
                f'reanalysis-era5-{"single-levels" if variable_type == "surface" else "pressure-levels"}',
                request,
                os.path.join(output_path, file_name)
            )

In [ ]:
# Loop through each year from 2019 to 2022
for year in range(1992, 1993):
    # Loop through each country in the dictionary
    for country_code, country_name in countries_dict.items():
        area = country_areas[country_code]
        # Initialize the progress bar
        with tqdm(total=len(surface_variables + pressure_level_variables),
                  desc=f"Downloading data for {country_name} {year}") as pbar:
            retry_count = 0
            success = False
            while retry_count < 100 and not success:
                try:
                    # Call the download function
                    download_data(str(year), country_code, country_name, area, output_directory)
                    # Update the progress bar after each completion
                    pbar.update(len(surface_variables + pressure_level_variables))
                    success = True
                except Exception as e:
                    retry_count += 1
                    print(f"Error downloading data for {country_name} {year}: {e}")
                    print(f"Retrying {retry_count}/100...")
                    time.sleep(5)  # Wait for 5 seconds before retrying
            if not success:
                print(f"Failed to download data for {country_name} {year} after 3 attempts.")
                break  # Stop the script if retries fail

2025-01-29 08:33:38,656 INFO [2025-01-09T00:00:00] Please be aware that ERA5 data from 1st January 2025 was degraded and is being corrected. Watch the [Forum announcement](https://forum.ecmwf.int/t/era5-data-from-1st-january-2025-was-degraded-and-is-being-corrected/10689) for updates.
2025-01-29 08:33:38,657 INFO Request ID is b75e859f-55f8-412d-bb93-43f0f351f131
2025-01-29 08:33:38,938 INFO status has been updated to accepted
2025-01-29 09:41:53,169 INFO status has been updated to running
2025-01-29 09:46:01,722 INFO status has been updated to successful


c6c1b8e61aa2307f9174a17b8fa90a0c.nc:   0%|          | 0.00/33.6M [00:00<?, ?B/s]

2025-01-29 09:46:11,042 INFO [2025-01-09T00:00:00] Please be aware that ERA5 data from 1st January 2025 was degraded and is being corrected. Watch the [Forum announcement](https://forum.ecmwf.int/t/era5-data-from-1st-january-2025-was-degraded-and-is-being-corrected/10689) for updates.
2025-01-29 09:46:11,044 INFO Request ID is 007eb2f8-a141-4c0c-a5e1-49c0d55220a0
2025-01-29 09:46:11,270 INFO status has been updated to accepted
2025-01-29 09:52:42,180 INFO status has been updated to running
2025-01-29 09:56:46,171 INFO status has been updated to successful


b195ee4ca760ce4540bdcac4f7756d7e.nc:   0%|          | 0.00/34.6M [00:00<?, ?B/s]

2025-01-29 09:56:53,887 INFO [2025-01-09T00:00:00] Please be aware that ERA5 data from 1st January 2025 was degraded and is being corrected. Watch the [Forum announcement](https://forum.ecmwf.int/t/era5-data-from-1st-january-2025-was-degraded-and-is-being-corrected/10689) for updates.
2025-01-29 09:56:53,888 INFO Request ID is 74d8ddc4-193b-47d5-81f8-80d51ab54f15
2025-01-29 09:56:54,752 INFO status has been updated to accepted
2025-01-29 10:34:20,572 INFO status has been updated to running
2025-01-29 10:38:23,358 INFO status has been updated to successful


...

2025-01-29 14:07:24,877 INFO [2025-01-09T00:00:00] Please be aware that ERA5 data from 1st January 2025 was degraded and is being corrected. Watch the [Forum announcement](https://forum.ecmwf.int/t/era5-data-from-1st-january-2025-was-degraded-and-is-being-corrected/10689) for updates.
2025-01-29 14:07:24,878 INFO Request ID is 0808e7eb-dee5-4953-8998-fe967adcb764
2025-01-29 14:07:25,071 INFO status has been updated to accepted
2025-01-29 14:15:56,943 INFO status has been updated to running
2025-01-29 14:20:01,603 INFO status has been updated to successful


b3f3b042af4209ab467f813a0791f8db.nc:   0%|          | 0.00/28.8M [00:00<?, ?B/s]

2025-01-29 14:20:10,325 INFO [2025-01-09T00:00:00] Please be aware that ERA5 data from 1st January 2025 was degraded and is being corrected. Watch the [Forum announcement](https://forum.ecmwf.int/t/era5-data-from-1st-january-2025-was-degraded-and-is-being-corrected/10689) for updates.
2025-01-29 14:20:10,325 INFO Request ID is 7cf88731-7c67-4a75-9aab-808260272a8a
2025-01-29 14:20:10,592 INFO status has been updated to accepted
2025-01-29 14:28:47,905 INFO status has been updated to running
2025-01-29 14:32:52,585 INFO status has been updated to successful


706c7a763b6880f03f77aa9a52b49191.nc:   0%|          | 0.00/4.03M [00:00<?, ?B/s]

2025-01-29 14:32:58,709 INFO [2025-01-09T00:00:00] Please be aware that ERA5 data from 1st January 2025 was degraded and is being corrected. Watch the [Forum announcement](https://forum.ecmwf.int/t/era5-data-from-1st-january-2025-was-degraded-and-is-being-corrected/10689) for updates.
2025-01-29 14:32:58,711 INFO Request ID is d6994055-1852-403c-9288-0cb22a82e278
2025-01-29 14:32:58,896 INFO status has been updated to accepted
2025-01-29 14:41:27,045 INFO status has been updated to running


In [ ]:
# Loop through each country and download data
for country_code, country_name in countries_dict.items():
    area = country_areas[country_code]
    with tqdm(total=len(surface_variables + pressure_level_variables), desc=f"Downloading {country_name}") as pbar:
        download_data("2021", country_code, country_name, area, output_directory)
        pbar.update(len(surface_variables + pressure_level_variables))

In [ ]:
# Loop through each year from 2010 to 2015
for year in range(2016, 2023):
     #Loop through each country in the dictionary
    for country_code, country_name in countries_dict.items():
        area = country_areas[country_code]
         #Initialize the progress bar
        with tqdm(total=len(surface_variables + pressure_level_variables),
                  desc=f"Downloading data for {country_name} {year}") as pbar:
             #Call the download function
            download_data(str(year), country_code, country_name, area, output_directory)
            # Update the progress bar after each completion
            pbar.update(len(surface_variables + pressure_level_variables))
